In [1]:
import tomica
print(tomica.__file__)

/Users/zhangyangfa/rust-projects/tomica/tomica/__init__.py


In [2]:
import polars as pl
import random
import time

In [3]:
data = pl.select(
    pl.lit(1).sample(n=1e6, with_replacement=True)  # 先采样
    .map_elements(lambda _: random.uniform(-50, 50), return_dtype=pl.Float64)
    .alias("foo")
).to_series()

In [4]:
def test_speed(fn, window_size):
    st = time.time()
    try:
        fn(data, window_size)
        print(f'{fn.__name__:<20} {time.time()-st:>8.4f} s')
    except:
        fn(data, data, window_size)
        print(f'{fn.__name__:<20} {time.time()-st:>8.4f} s')

In [6]:
for func in tomica.operators.__all__:
    func = getattr(tomica.operators,func)
    test_speed(func,10)

ts_delay               0.0007 s
ts_delta               0.0014 s
ts_returns             0.0029 s
ts_sum                 0.0159 s
ts_product             0.0126 s
ts_max                 0.0155 s
ts_min                 0.0138 s
ts_mean                0.0092 s
ts_std                 0.0179 s
ts_rank                0.0915 s
ts_variance            0.0162 s
ts_quantile_up         0.0303 s
ts_quantile_down       0.0299 s
ts_zscore              0.0266 s
ts_robust_zscore       0.0717 s
ts_scale               0.0258 s
ts_sharpe              0.0271 s
ts_av_diff             0.0087 s
ts_min_diff            0.0131 s
ts_max_diff            0.0125 s
ts_cov                 0.0272 s
ts_beta                0.0470 s
ts_pos_count           0.0037 s
ts_neg_count           0.0040 s
decay_n                0.0027 s
